In [1]:
import numpy as np
from scipy.stats import norm
from numpy.polynomial.polynomial import Polynomial
import matplotlib.pyplot as plt
from tqdm import tqdm

## 1D Bermudan Call Longstaff Schwartz


The Longstaff-Schwartz algorithm is a Monte Carlo simulation method used for pricing American-style options. It combines path simulation with regression techniques to estimate the early exercise boundary and option prices.

For a 1-dimensional Bermudan Max-Call Option, which is fundamentally a Bermudan Call Option, the initial value is specified by:

$$
V_0(X) = \sup_{\tau} \mathbb{E}\left[\frac{h(X_{\tau})}{B_{\tau}}\right]
$$
where $h(X_t) = (X_t-K)^{+}$ is the payoff for 1-dimensional Max-Call Option at time $t$, and $B_\tau = e^{\tau r}$ is the discout factor between $t=0$ and $t=\tau$.

We implement the algorithm by backward induction, from the option's maturity back to the initial time. At each stopping time, the algorithm decides whether to exercise the option or continue holding it.

$$
V_{\text{maturity}} = \max(S_{\text{maturity}} - K, 0)
$$

For each stpping in $t_{M-1}, t_{M-2} \ldots, t_1$, perform the following steps:


1. Identify in-the-money paths where the option's immediate exercise value is positive. This is to improve accuracy as discussed in the lecture.
   
2. Use polynomial regression to fit the continuation value as a function of the underlying asset's price at time $t_i$ for in-the-money paths. The regression is based on the discounted option values at time $t_{i+1}$. To demonstrate, we estimate
the continuation value
$$
C_{t_{i}}(x) = \frac{B_{t_i}}{B_{t_{i+1}}}\mathbb{E}[V_{t_{i+1}}(X_{t_{i+1}}) \mid X_{t_{i}} = x]
$$
as sum of $R$ basis function on the underlying price $X_{t_{i}} = x$:
$$
\hat{C}_{t_{i}}(x) = \sum_{r=1}^{R} \beta_r \psi_r (x)
$$

   
3. Calculate the continuation value using the regression model for all in-the-money paths.
   
4. Compare the immediate exercise value, $h(X_{t_i})$, with the estimated continuation value for each in-the-money path.
   
5. Update the option value, $V_{t_i} = \max\{h(X_{t_i}), V_{t_{i+1}}\}$ 

The estimated price of the option is obtained by averaging the discounted option values at the first stopping time and discounting back to the present:

$$
\text{Option Price} = \mathbb{E}[e^{-r\Delta t} V_{t_1}]
$$
where $\Delta t = \frac{T}{M}$ is the time interval between exercise date assuming equidistant exercise.



In [2]:
def simulate_gbm(S0, r, dividend, sigma, T, M, I, seed = None):
    """
    Generates Geometric Brownian Motion paths for one underlying asset.
    Parameters:
        S0 : float : initial stock price
        r : float : risk-free rate
        dividend : float : dividend yield
        sigma : float : volatility
        T : float : time to maturity
        M : int : number of stopping times, in our case it is 9
        I : int : number of paths
        seed: int : seed for random number generator
    Returns:
        numpy.ndarray : simulated paths of dimension (M + 1, I)
    """
    dt = T / M
    paths = np.zeros((M + 1, I))
    paths[0] = S0
    
    if seed is not None:
        np.random.seed(seed)
    # Generate paths
    for t in range(1, M + 1):
        Z = np.random.standard_normal(I)   
        paths[t] = paths[t - 1] * np.exp((r-dividend - 0.5 * sigma ** 2) * dt + sigma * np.sqrt(dt) * Z)
    return paths

def longstaff_schwartz_polyfit_degree_train(S_paths_train, K, r, T, degree):
    """
    Trains the Longstaff-Schwartz algorithm using polynomial regression and return the coefficients of the polynomial regression.
    Parameters:
        S_paths_train : numpy.ndarray : simulated paths of the underlying asset of shape (M + 1, I)
        K : float : strike price
        r : float : risk-free rate
        T : float : time to maturity
        degree : int : degree of the polynomial regression
    Returns:
        list : list of polynomial coefficients of length M
    """
    M, I = S_paths_train.shape
    M -= 1 # number of stopping times excluding initial time
    dt = T / M
    discount = np.exp(-r * dt) # 1 step discount factor
    
    V = np.maximum(S_paths_train[-1] - K, 0) # option value at maturity
    coeffs = []
    
    for t in range(M - 1, 0, -1): # i.e. t = M - 1, M - 2, ..., 1
        in_the_money = S_paths_train[t] > K
        X = S_paths_train[t, in_the_money] # only regress on in-the-money paths lead to improved accuracy
        Y = V[in_the_money] * discount # discounted option value
        if len(X) == 0: # if there are no in-the-money paths, just continue to next iteration
            coeffs.append(None)
            continue
        coeffs.append(np.polyfit(X, Y, degree)) # store the coefficients of the polynomial regression
        continuation_value = np.polyval(coeffs[-1], S_paths_train[t]) # C^hat for all paths, shape (I,)
        immediate_exercise_value = np.maximum(S_paths_train[t] - K, 0) # Immediate payoff for all paths, shape (I,)
        # based on Longstaff & Schwartz, 2001, set V_t-1 = V_t if immediate exercise value > continuation value
        V = np.where(immediate_exercise_value > continuation_value, immediate_exercise_value, V)
    return coeffs[::-1] # reverse the list of coefficients because we are going backward in time

def longstaff_schwartz_polyfit_degree_test(S_paths_test, K, r, T, degree, coeffs):
    """
    Tests the Longstaff-Schwartz algorithm using polynomial regression and return the estimated price of the option.
    Parameters:
        S_paths_test : numpy.ndarray : simulated paths of the underlying asset of shape (M + 1, I)
        K : float : strike price
        r : float : risk-free rate
        T : float : time to maturity
        degree : int : degree of the polynomial regression
        coeffs : list : list of polynomial coefficients of length M
    Returns:
        float : estimated price of the option
    """
    M, I = S_paths_test.shape
    M -= 1
    dt = T / M
    discount = np.exp(-r * dt)
    
    V = np.maximum(S_paths_test[-1] - K, 0)
    
    for t in range(M - 1, 0, -1):
        if coeffs[t-1] is not None:
            continuation_value = np.polyval(coeffs[t-1], S_paths_test[t])
            immediate_exercise_value = np.maximum(S_paths_test[t] - K, 0)
            V = np.where(immediate_exercise_value > continuation_value, immediate_exercise_value, V)
    
    return np.mean(V) * discount # discount from t1 to t0

In [3]:
# Parameters
S0 = 100 # initial stock price
r = 0.05 # risk-free rate
dividend = 0.1 # dividend yield
sigma = 0.2 # symmetric volatility
K = 100 # strike price
degree = 5 # degree of the polynomial regression
T = 3 # time to maturity
M = 9 # number of stopping times
d = 1
I_train = (1500+d) * 8192 # same number of paths as in paper
seed_train = 1 # seed for training
# Train the Longstaff-Schwartz algorithm
S_train = simulate_gbm(S0, r, dividend, sigma, T, M, I_train, seed = seed_train)
coeffs = longstaff_schwartz_polyfit_degree_train(S_train, K, r, T, degree)

In [4]:
# Use Longstaff-Schwartz algorithm for pricing
seed_test = 2 # seed for testing
I_test = 100000 # number of paths for testing
S_test = simulate_gbm(S0, r, dividend, sigma, T, M, I_test, seed = seed_test)
price = longstaff_schwartz_polyfit_degree_test(S_test, K, r, T, degree, coeffs)
print(f"Estimated price of the 1D Bermudan Max Call Option: {price:.2f}")

Estimated price of the 1D Bermudan Max Call Option: 8.14


## Multi-Dimensional Longstaff schwartz

For a Bermudan Max-Call Option on $S_1,S_2, \dots, S_d$ , the payoff is given by: $$\sup_{\tau} \mathbb{E}\left[ e^{-r\tau} \left( \max_{1 \leq i \leq d} S_{\tau}^i - K \right)^+ \right]$$
The Longstaff-Schwartz method still work similarly to the 1-dimensional case, except that in each stopping time of the backward induction, we need to represent $\hat{S_t} = \max\{S_t^1, S_t^2, \dots, S_t^d\}$ as the effective underlying for polynomial regression, while the other steps are the same as before.

In [5]:
def GBM(d, mu, sigma, S0, T, dt, number_of_paths, seed=None):
    """
    Efficiently simulates number_of_paths many d-dimensional geometric brownian motion (GBM) sample paths.

    Arguments:
    d : int
        The dimension of the GBM to be simulated.
    mu : array
        Drift values in an array of shape (d,).
    sigma : array
        Volatilities in an array of shape (d,).
    S0 : array
        Initial values of the GBM in an array of shape (d,).
    T : float
        Specifies the time interval [0, T] in which the GBM will be simulated.
    dt : float
        Specifies the time increments.
    number_of_paths : int
        Number of sample paths to be simulated.
    seed : int, optional
        The seed for the random number generator to ensure reproducibility.

    Returns:
    A numpy array of GBM simulations of shape (number_of_paths, d, n) where n = T/dt, more memory efficient.
    """
    if seed is not None:
        np.random.seed(seed)  # Set the seed if provided

    n = int(T / dt)  # number of time steps
    dt_sqrt = np.sqrt(dt)
    
    # Precompute drift and diffusion terms
    drift_term = (mu - 0.5 * sigma**2) * dt
    diffusion_term = sigma * dt_sqrt
    
    # Initialize the simulations array
    S = np.empty((number_of_paths, d, n + 1), dtype=np.float32)
    S[:, :, 0] = S0

    # Simulate paths
    for t in range(1, n + 1):
        Z = np.random.randn(number_of_paths, d).astype(np.float32)
        S[:, :, t] = S[:, :, t-1] * np.exp(drift_term + diffusion_term * Z) # exact solution of GBM

    return S

def g(x, k):
  """
  The  payoff function of Bermudan Max Call Option across all paths at all time at each time, this is not discounted
  """
  y = np.maximum(np.amax(x - k, axis = 1), 0) #max(S1,...,Sd) - k at each time step
  return y # g = (max(S1,...,Sd) - k)^+

def longstaff_schwartz_polyfit_degree_train(S_paths_train, K, r, T, degree):
    I, N = S_paths_train.shape
    N -= 1
    dt = T / N
    discount = np.exp(-r * dt)
    
    V = np.maximum(S_paths_train[:, -1] - K, 0)
    coeffs = []
    
    for t in range(N - 1, 0, -1):
        in_the_money = S_paths_train[:, t] > K
        X = S_paths_train[in_the_money, t]
        Y = V[in_the_money] * discount
        if len(X) == 0:
            coeffs.append(None)
            continue
        coeffs.append(np.polyfit(X, Y, degree))
        continuation_value = np.polyval(coeffs[-1], S_paths_train[:, t])
        immediate_exercise_value = np.maximum(S_paths_train[:, t] - K, 0)
        # if immediate exercise value is higher than continuation value, exercise, otherwise use continuation value
        V = np.where(immediate_exercise_value > continuation_value, immediate_exercise_value, V)
    
    return coeffs[::-1] # reverse the list of coefficients

def longstaff_schwartz_polyfit_degree_test(S_paths_test, K, r, T, degree, coeffs):
    I, N = S_paths_test.shape
    N -= 1
    dt = T / N
    discount = np.exp(-r * dt)
    
    V = np.maximum(S_paths_test[:, -1] - K, 0)
    
    for t in range(N - 1, 0, -1):
        if coeffs[t-1] is not None:
            continuation_value = np.polyval(coeffs[t-1], S_paths_test[:, t])
            immediate_exercise_value = np.maximum(S_paths_test[:, t] - K, 0)
            V = np.where(immediate_exercise_value > continuation_value, immediate_exercise_value, V)
    
    return np.mean(V) * discount


In [6]:
r = 0.05  #interest rate
dividend = 0.1  #dividend rate
k = 100.0    #Strike price
T = 3.0    #final time
N = 9  # number of stopping times
dt = T/N  #time intervals
I_test = 100000 # number of paths for testing
seed_train = 1 # train seed
seed_test = 2 # test seed
degree = 5  #degree of polynomial regression
prices = {}
for d in tqdm([2, 5, 10], desc = "Simulating"):
    I_train = (1500+d) * 8192
    mu = (r - dividend) * np.ones(shape = (d,))
    sigma = 0.2 * np.ones(shape = (d,))
    S0 = 100.0 * np.ones(shape = (d,))
    
    X_train = GBM(d, mu, sigma, S0, T, dt, I_train, seed = seed_train)
    effective_X_train = np.amax(X_train, axis = 1)
    coeffs = longstaff_schwartz_polyfit_degree_train(effective_X_train, k, r, T, degree)
    
    # Assuming the function `longstaff_schwartz_price` computes the price
    X_test = GBM(d, mu, sigma, S0, T, dt, I_test, seed = seed_test)
    effective_X_test = np.amax(X_test, axis = 1) #max(S1,...,Sd) at each time step
    price = longstaff_schwartz_polyfit_degree_test(effective_X_test, k, r, T, degree, coeffs)
    prices[d] = price

Simulating:   0%|          | 0/3 [00:00<?, ?it/s]/var/folders/nm/7d2mnp_x4y1c6q58d9_5h1140000gn/T/ipykernel_29488/2597674345.py:70: RankWarning: Polyfit may be poorly conditioned
  coeffs.append(np.polyfit(X, Y, degree))
/var/folders/nm/7d2mnp_x4y1c6q58d9_5h1140000gn/T/ipykernel_29488/2597674345.py:70: RankWarning: Polyfit may be poorly conditioned
  coeffs.append(np.polyfit(X, Y, degree))
/var/folders/nm/7d2mnp_x4y1c6q58d9_5h1140000gn/T/ipykernel_29488/2597674345.py:70: RankWarning: Polyfit may be poorly conditioned
  coeffs.append(np.polyfit(X, Y, degree))
/var/folders/nm/7d2mnp_x4y1c6q58d9_5h1140000gn/T/ipykernel_29488/2597674345.py:70: RankWarning: Polyfit may be poorly conditioned
  coeffs.append(np.polyfit(X, Y, degree))
/var/folders/nm/7d2mnp_x4y1c6q58d9_5h1140000gn/T/ipykernel_29488/2597674345.py:70: RankWarning: Polyfit may be poorly conditioned
  coeffs.append(np.polyfit(X, Y, degree))
/var/folders/nm/7d2mnp_x4y1c6q58d9_5h1140000gn/T/ipykernel_29488/2597674345.py:70: RankWarn

In [7]:
for d, price in prices.items():
    print(f"Dimension {d}: Price {price.item():.4f}")

Dimension 2: Price 14.2297
Dimension 5: Price 27.3789
Dimension 10: Price 41.3548
